<a href="https://colab.research.google.com/github/AnnLivio/powerbi/blob/main/Pizza_sales_sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We ♥ Pizza
## Pizza Sales Analysis 🍕

### **Orders** Table

|Field|Description|
|---|---|
|order\_id|Unique identifier|
|date|Date the order was placed|
|time|Time the order was placed|


---

### **Order details** Table

* **order\_details\_id:** Unique identifier for each pizza
* **order\_id:** Foreign key that ties the details in each order to the order itself
* **pizza\_id:** Foreign key that ties the pizza ordered to its details, like size and price
* **quantity:** Quantity ordered for each pizza of the same type and size

---

### **Pizza** Table
* **pizza\_id:** Unique identifier for each pizza \(constituted by its type and size\)
* **pizza\_type\_id:** Foreign key that ties each pizza to its broader pizza type
* **size:** Size of the pizza \(Small, Medium, Large, X Large, or XX Large\)
* **price:** Price of the pizza in USD


---

### **Pizza_type** Table

* **pizza_type_id**: Unique identifier for each pizza type
* **name:** Name of the pizza as shown in the menu
* **category:** Category in the menu (Classic, Chicken, Supreme, or Veggie)
* **ingredients:** Comma-delimited ingredients as shown in the menu

## Import libraries

In [32]:
# Import libraries and packages
import pandas as pd
import sqlite3

## Create a database, connection and cursor

In [33]:
con = sqlite3.connect('pizzasales.db')
cur = con.cursor()

In [50]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [35]:
%sql sqlite:///pizzasales.db

In [39]:
# My salvation!!!
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

## Create tables

In [40]:
table_list = ['orders', 'order_details', 'pizzas', 'pizza_types']

for table in table_list:
  path = f"/content/drive/MyDrive/Data Analysis/Simple_Projects/pizza_sales_data/{table}.csv"
  df = pd.read_csv(path, encoding="latin-1")
  df.to_sql(table, con, if_exists='replace', index=False, method=None)
  print(table)


orders
order_details
pizzas
pizza_types


## Look tables

In [41]:
%sql pragma table_info(orders)

 * sqlite:///pizzasales.db
Done.


cid,name,type,notnull,dflt_value,pk
0,order_id,INTEGER,0,None,0
1,date,TEXT,0,None,0
2,time,TEXT,0,None,0


In [42]:
%sql pragma table_info(order_details)

 * sqlite:///pizzasales.db
Done.


cid,name,type,notnull,dflt_value,pk
0,order_details_id,INTEGER,0,None,0
1,order_id,INTEGER,0,None,0
2,pizza_id,TEXT,0,None,0
3,quantity,INTEGER,0,None,0


In [43]:
%sql pragma table_info(pizza_types)

 * sqlite:///pizzasales.db
Done.


cid,name,type,notnull,dflt_value,pk
0,pizza_type_id,TEXT,0,None,0
1,name,TEXT,0,None,0
2,category,TEXT,0,None,0
3,ingredients,TEXT,0,None,0


In [44]:
%sql pragma table_info(pizzas)

 * sqlite:///pizzasales.db
Done.


cid,name,type,notnull,dflt_value,pk
0,pizza_id,TEXT,0,None,0
1,pizza_type_id,TEXT,0,None,0
2,size,TEXT,0,None,0
3,price,REAL,0,None,0


# Orders and Order_details tables

In [46]:
%sql SELECT * FROM orders LIMIT 5

 * sqlite:///pizzasales.db
Done.


order_id,date,time
1,2015-01-01,11:38:36
2,2015-01-01,11:57:40
3,2015-01-01,12:12:28
4,2015-01-01,12:16:31
5,2015-01-01,12:21:30


In [48]:
%sql SELECT strftime('%d',date) FROM orders LIMIT 5

 * sqlite:///pizzasales.db
Done.


"strftime('%d',date)"
01
01
01
01
01


# Pizzas tables

## Create a view

In [ ]:
CREATE VIEW pizzas_full AS
SELECT
  pizza_id,
  price,
  size,
  pt.name,
  pt.category
FROM pizzas
INNER JOIN pizza_types pt ON pt.pizza_type_id = pizzas.pizza_type_id;

## CTEs

In [55]:
# Create view table for pizzas and pizza types
%%sql
WITH pizzasfull AS (
  SELECT
    pizza_id,
    price,
    size,
    pt.name,
    pt.category
  FROM pizzas
  INNER JOIN pizza_types pt ON pt.pizza_type_id = pizzas.pizza_type_id
  ),
  ordersfull AS (
    SELECT
      od.order_id,
      date,
      strftime('%H', time) AS hour,
      od.pizza_id,
      od.quantity
    FROM orders
    INNER JOIN order_details od ON od.order_id = orders.order_id
  )
SELECT order_id, date, hour, quantity, pizzasfull.price, pizzasfull.size, pizzasfull.name, pizzasfull.category
FROM ordersfull
LEFT JOIN pizzasfull USING(pizza_id)
LIMIT 5
;

 * sqlite:///pizzasales.db
Done.


order_id,date,hour,quantity,price,size,name,category
1,2015-01-01,11,1,13.25,M,The Hawaiian Pizza,Classic
2,2015-01-01,11,1,16.0,M,The Classic Deluxe Pizza,Classic
2,2015-01-01,11,1,18.5,L,The Five Cheese Pizza,Veggie
2,2015-01-01,11,1,20.75,L,The Italian Supreme Pizza,Supreme
2,2015-01-01,11,1,16.0,M,The Mexicana Pizza,Veggie


## **Questions**

### 1 How many customers do we have each day? Are there any peak hours?

In [77]:
# How many customers do we have each day?
%sql SELECT ROUND(AVG(customers),0) FROM (SELECT COUNT(order_id) AS customers FROM orders GROUP BY date);

 * sqlite:///pizzasales.db
Done.


"ROUND(AVG(customers),0)"
60.0


In [72]:
# Are there any peak hours?
%%sql
SELECT
    date,
    strftime('%H', time) AS hour,
    SUM(od.quantity) AS total_qty
  FROM orders
  INNER JOIN order_details od ON od.order_id = orders.order_id
  GROUP BY date, hour
  ORDER BY total_qty DESC LIMIT 10;

 * sqlite:///pizzasales.db
Done.


date,hour,total_qty
2015-10-15,13,64
2015-01-08,12,49
2015-06-04,12,49
2015-07-30,13,48
2015-02-03,12,47
2015-12-14,12,47
2015-06-01,12,46
2015-11-18,12,46
2015-10-01,12,45
2015-08-14,13,44


### 2. How many pizzas are typically in an order? Do we have any bestsellers?

In [65]:
# Average pizzas per order
%%sql
WITH fullorders AS (
  SELECT
      od.order_id,
      SUM(od.quantity) AS total_qty
  FROM orders
  INNER JOIN order_details od ON od.order_id = orders.order_id
  GROUP BY od.order_id
)
SELECT ROUND(AVG(total_qty),2) as 'AVG Quantity' FROM fullorders;

 * sqlite:///pizzasales.db
Done.


AVG Quantity
2.32


In [97]:
# Bestseller pizza
%%sql
WITH fullpizzas AS (
  SELECT
    pizza_id,
    pt.name as name
  FROM pizzas
  INNER JOIN pizza_types pt ON pt.pizza_type_id = pizzas.pizza_type_id
)
SELECT
    fp.name,
    SUM(quantity) AS total_qty
  FROM order_details od
  INNER JOIN fullpizzas fp ON fp.pizza_id = od.pizza_id
  GROUP BY fp.name
  ORDER BY total_qty DESC LIMIT 3;


 * sqlite:///pizzasales.db
Done.


name,total_qty
The Classic Deluxe Pizza,2453
The Barbecue Chicken Pizza,2432
The Hawaiian Pizza,2422


### 3. How much money did we make this year? Can we indentify any seasonality in the sales?

In [99]:
# How much money did we make this year?
%%sql
WITH subtotals AS (
  SELECT
    od.pizza_id,
    (od.quantity * p.price) AS subtotal

  FROM order_details od, pizzas p
  WHERE od.pizza_id = p.pizza_id
  )
  SELECT ROUND(SUM(subtotal),2) AS year_revenue FROM subtotals;

 * sqlite:///pizzasales.db
Done.


year_revenue
817860.05


In [107]:
# Can we indentify any seasonality in the sales?
%%sql
WITH sales_per_month AS (
  SELECT strftime('%m', date) as month, COUNT(order_id) as total_orders
  FROM orders
  GROUP BY month
)
SELECT CASE month
          WHEN '01' THEN 'January'
          WHEN '02' THEN 'February'
          WHEN '03' THEN 'March'
          WHEN '04' THEN 'April'
          WHEN '05' THEN 'May'
          WHEN '06' THEN 'June'
          WHEN '07' THEN 'July'
          WHEN '08' THEN 'August'
          WHEN '09' THEN 'September'
          WHEN '10' THEN 'October'
          WHEN '11' THEN 'November'
           ELSE 'December'
       END AS month,
       total_orders
FROM sales_per_month
ORDER BY total_orders DESC;

 * sqlite:///pizzasales.db
Done.


month,total_orders
July,1935
May,1853
January,1845
August,1841
March,1840
April,1799
November,1792
June,1773
February,1685
December,1680


### 4. Are there any pizzas we should take of the menu, or any promotions we could leverage?

In [108]:
# worstseller pizza
%%sql
WITH fullpizzas AS (
  SELECT
    pizza_id,
    pt.name as name
  FROM pizzas
  INNER JOIN pizza_types pt ON pt.pizza_type_id = pizzas.pizza_type_id
)
SELECT
    fp.name,
    SUM(quantity) AS total_qty
  FROM order_details od
  INNER JOIN fullpizzas fp ON fp.pizza_id = od.pizza_id
  GROUP BY fp.name
  ORDER BY total_qty LIMIT 3;

 * sqlite:///pizzasales.db
Done.


name,total_qty
The Brie Carre Pizza,490
The Mediterranean Pizza,934
The Calabrese Pizza,937


In [79]:
#